In [1]:
from collections import Counter
from tqdm import tqdm
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

import matplotlib.pyplot as plt
%matplotlib inline

import gensim.models
from gensim.test.utils import datapath

import MeCab
import ipadic

/home/kawahara/.conda/envs/sds/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
b_tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
b_model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
m_tokenizer = MeCab.Tagger(ipadic.MECAB_ARGS)
cm = gensim.models.KeyedVectors.load_word2vec_format("assets/cc.ja.300.vec.gz")

In [210]:
def masking_text(text, mecab_tokenizer, bert_tokenizer):

    """文を受け取りMeCabとBERTトークナイザの結果からマスク位置を決定，マスクされた文のリストを出力"""
    
    mask_labels = [] # ['r:そのまま', 'm:マスクする', 'h:まとめてマスクする', 's:まとめられる語']
    ogl_mecab_tokens = []
    masked_texts = []
    target_tokens = []
    zure = 0
    
    node = mecab_tokenizer.parseToNode(text)
    tokenized_text = bert_tokenizer.tokenize(text)
    
    node = node.next #文頭，文末形態素を無視
    while node.next:
        
        #print(node.feature)
        
        if node.surface not in  tokenized_text: #サブワード化された語を変換対象から外すため
            mask_labels.append('r')
        elif node.feature.split(',')[0] in ['助詞', '助動詞', '連体詞', '形容詞', '記号']:
            mask_labels.append('r')
        elif node.feature.split(',')[1] == 'サ変接続':
            mask_labels.append('h')
        elif node.feature.split(',')[4] == 'サ変・スル' and mask_labels[-1] == 'h':
            mask_labels.append('s')
        else:
            mask_labels.append('m')
        ogl_mecab_tokens.append(node.surface)
        node = node.next
        
    #print(mask_labels)
    
    memory_masked = ogl_mecab_tokens[:]
    
    for target_idx, mask_label in enumerate(mask_labels):
        mecab_tokens = ogl_mecab_tokens[:]
        
        target_token = mecab_tokens[target_idx]
        
        if mask_label == 'm':
            mecab_tokens[target_idx] = '[MASK]'
            memory_masked[target_idx-zure] = '[MASK]'
        elif mask_label == 'h':
            mecab_tokens[target_idx] = '[MASK]'
            memory_masked[target_idx-zure] = '[MASK]'
            for m_label in mask_labels[target_idx+1:]:
                if m_label == "s":
                    mecab_tokens.pop(target_idx+1)
                    memory_masked.pop(target_idx+1)
                    zure = zure + 1
                else:
                    break
        elif mask_label in ['r','s']:
            continue
        
        masked_text = "".join(mecab_tokens)
        
        masked_texts.append(masked_text)
        target_tokens.append(target_token)
        
        mecab_tokens[target_idx] = target_token
        
    
    return masked_texts, target_tokens, memory_masked

In [211]:
def get_candidates(text, masked_text, bert_tokenizer, bert_model):
    
    """マスクされた文と原文を受け取って候補を10語出力"""
    
    encoded_dict = bert_tokenizer(text, masked_text)

    masked_idx = encoded_dict["input_ids"].index(4)
    tokens_tensor = torch.tensor([encoded_dict["input_ids"]])
    segments_tensors = torch.tensor([encoded_dict["token_type_ids"]])

    bert_model.eval()
    if torch.cuda.is_available():
        tokens_tensor = tokens_tensor.to('cuda')
        segments_tensors = segments_tensors.to('cuda')
        bert_model.to('cuda')
    with torch.no_grad():
        outputs = bert_model(tokens_tensor, token_type_ids=segments_tensors)
        predictions = outputs[0]

    topk_score, topk_index = torch.topk(predictions[0, masked_idx], 10)
    topk_tokens = bert_tokenizer.convert_ids_to_tokens(topk_index.tolist())
    
    bert_rank = np.array([i for i in range(len(topk_tokens))])
    
    topk_data = {"tokens": topk_tokens, "score": topk_score, "index": topk_index, "rank": bert_rank}
    
    return topk_data

In [212]:
def frequency_ranking(candidates):
    
    """頻度表を検索，高い順からランキングします"""
    
    import pickle
    import os
    
    if os.path.isfile('assets/bccwj_frequency.pickle') == 'True':
        bccwj_frequency = {}
        with open('assets/BCCWJ_goihyo_utf8.txt') as BCCWJ:
            for BCCWJ_line in BCCWJ:
                BCCWJ_line_list = BCCWJ_line.split('\t')
                if BCCWJ_line_list[0] != 'ID_BCCWJ':
                    bccwj_frequency[BCCWJ_line_list[3]] = sum(int(i) for i in BCCWJ_line_list[9:15])
        
        with open("assets/bccwj_frequency.pickle","wb") as f:
            pickle.dump(bccwj_frequency, f)
        
    with open('assets/bccwj_frequency.pickle', 'rb') as f:
        bccwj_frequency = pickle.load(f)
    
    candidates_frequency = [bccwj_frequency.get(cand, 0) for cand in candidates]
    frequency_rank = np.array([np.argsort([-int(freq) for freq in candidates_frequency]).tolist().index(idx) for idx in range(10)]) #ランク付けの呪文
    
    frequency_data = {"frequency":candidates_frequency, "rank":frequency_rank,}
    
    return frequency_data

In [213]:
def fasttext_similarity_ranking(target_token, candidates, fasttext_vec):
    
    """Cos類似度を求めてランク付けする．gensimで読み込んでるんだったら他も使えるかも"""
    
    cosine_sim = [cm.similarity(target_token, cand) if target_token in cm and cand in cm else 0 for cand in candidates]
    
    sim_rank = np.array([np.argsort([-sim for sim in cosine_sim]).tolist().index(idx) for idx in range(10)]) #ランク付けの呪文
    
    sim_data = {"cos_sim": cosine_sim, "rank": sim_rank}
    
    return sim_data

In [233]:
def heiika(text):
    simple_words = []
    
    masked_list, target_tokens, memory_masked = masking_text(text, m_tokenizer, b_tokenizer)
    
    for text_idx, masked_text in enumerate(masked_list):
        topk_dict = get_candidates(text, masked_text, b_tokenizer, b_model)

        freq_data = frequency_ranking(topk_dict["tokens"])

        sim_dict = fasttext_similarity_ranking(target_tokens[text_idx], topk_dict["tokens"], cm)

        avg_idxs = np.argsort(topk_dict["rank"] + freq_data["rank"] + sim_dict["rank"]) #合算したランクが同じになったらBERTスコアが優先されます
        
        sorted_candidates = [topk_dict["tokens"][a_idx] for a_idx in avg_idxs]

        simple_words.append(sorted_candidates[0])

    kantannabun = []
    sw_idx = 0
    for tok in memory_masked:
        if tok == "[MASK]":
            kantannabun.append(simple_words[sw_idx])
            sw_idx = sw_idx + 1
        else:
            kantannabun.append(tok)

    print("".join(kantannabun))

In [234]:
heiika("七夕にきさらぎ駅へ訪れた")

七夕にきさらぎ駅へ訪れた
